#  Online Coupon Sales Analysis

Welcome to **Online Sales Coupon Analysis**.  

> **Crafted by:** Ashish Sahu |   
> **Powered by:** Pandas, Seaborn, Matplotlib, Numpy & Vibes 
---

In [6]:
#  Imports & Styling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import warnings
warnings.filterwarnings("ignore")

# Visualization Theme
sns.set(style="whitegrid")
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'axes.titlesize': 18,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})


In [8]:
#  Load Final CSV
df = pd.read_csv("C:\Online Sales Coupon Analysis\online_coupon_analysis_dataset.csv")
df['order_date'] = pd.to_datetime(df['order_date'])

# Backup raw
raw_df = df.copy()
print(" Dataset Loaded — Rows:", df.shape[0], "| Columns:", df.shape[1])


 Dataset Loaded — Rows: 10000 | Columns: 17


##  Dataset Breakdown

Let’s get comfy with what this dataset actually contains:

- **Customer-level** data (`customer_id`, `region`, `income`)
- **Sales transaction** data (`order_date`, `product_category`, `order_amount`)
- **Coupon data** (`coupon_type`, `discount_amount`, `coupon_used`)
- **Derived features**:
  - `final_amount`, `units_sold`, `discount_percentage`
  - `savings_per_unit`, `sales_category`, `month_name`

All ready for slicing, dicing, and insight-mining 

---


In [ ]:
#  Preview + Info
display(df.head())
print("\n Data Types & Null Check:")
print(df.info())
print("\n Missing Values:\n", df.isnull().sum())


##  Feature Engineering

-  `final_amount = order_amount - discount_amount`
-  `discount_percentage = (discount / order) * 100`
-  `savings_per_unit = discount / units_sold`
-  `sales_category` is a spend segment
-  `coupon_used`: 1 if applied, 0 if not

---

In [ ]:
#  Recalculate fields to verify
df['calc_discount_percent'] = np.round((df['discount_amount'] / df['order_amount']) * 100, 2)
df['calc_savings_per_unit'] = np.round(df['discount_amount'] / df['units_sold'], 2)

check_percent = np.allclose(df['calc_discount_percent'], df['discount_percentage'])
check_saving = np.allclose(df['calc_savings_per_unit'], df['savings_per_unit'])

print(" Discount % verified:", check_percent)
print(" Savings per unit verified:", check_saving)

# Clean temp cols
df.drop(['calc_discount_percent', 'calc_savings_per_unit'], axis=1, inplace=True)


##  Distribution of Coupon Types

Which kinds of coupons dominate the customer journey?

Let’s explore their frequency and get a feel for what customers are using the most.

---


In [ ]:
coupon_usage = df['coupon_type'].value_counts()

sns.barplot(x=coupon_usage.index, y=coupon_usage.values, palette='rocket')
plt.title("Coupon Type Distribution", fontsize=16)
plt.ylabel("Usage Count")
plt.xlabel("Coupon Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


##  Month-wise Sales Trend

Let’s visualize how total revenue shifts across months and years.  
This helps reveal **seasonal patterns**, **festival boosts**, and **planning windows**.

---


In [ ]:
monthly_sales = df.groupby(['year', 'month_name'])['final_amount'].sum().reset_index()
monthly_sales['month_num'] = monthly_sales['month_name'].apply(lambda x: list(calendar.month_name).index(x))
monthly_sales = monthly_sales.sort_values(['year', 'month_num'])

sns.lineplot(data=monthly_sales, x='month_name', y='final_amount', hue='year', marker="o", linewidth=2.5)
plt.title("Month-wise Total Revenue")
plt.ylabel("Revenue (₹)")
plt.xlabel("Month")
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


##  Product Category Breakdown

What are customers buying most?  
Let’s compare overall revenue, order size, and coupon savings by product category.

---


In [ ]:
prod_group = df.groupby('product_category')[['order_amount', 'discount_amount', 'final_amount']].sum().sort_values(by='final_amount', ascending=False)

prod_group.plot(kind='bar', stacked=True, colormap='coolwarm')
plt.title("Product Category Revenue Breakdown")
plt.ylabel("₹ Value")
plt.xlabel("Product Category")
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()


##  Region-wise Coupon Behavior

Let’s analyze which regions redeem coupons most.  
This can help marketers **target offers geographically**.

---


In [ ]:
region_coupon = df.groupby(['region', 'coupon_type']).size().unstack().fillna(0)

region_coupon.plot(kind='bar', stacked=True, colormap='Set2')
plt.title("Coupon Usage by Region")
plt.ylabel("Usage Count")
plt.xlabel("Region")
plt.xticks(rotation=0)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


##  Customer Income vs Spend

Do high-income customers spend more?  
Or do lower-income customers use coupons more aggressively?

We’ll explore with a scatterplot 

---


In [ ]:
sns.scatterplot(data=df, x='customer_income', y='final_amount', hue='sales_category', palette='husl')
plt.title("Customer Income vs Final Spend")
plt.xlabel("Monthly Income (₹)")
plt.ylabel("Final Spend (₹)")
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


##  Coupon Usage Heatmap (Month × Coupon Type)

Let’s visualize usage intensity over time for each coupon type.  
This shows which coupons pop in which months.

---


In [ ]:
heat_df = df.groupby(['month_name', 'coupon_type']).size().unstack().fillna(0)
heat_df = heat_df.reindex(list(calendar.month_name)[1:])  # Reorder months

sns.heatmap(heat_df, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title("Coupon Type Usage by Month")
plt.xlabel("Coupon Type")
plt.ylabel("Month")
plt.tight_layout()
plt.show()


##  Business KPI Dashboard

Here are the key performance indicators for the entire dataset:  
This gives us a high-level snapshot like any real dashboard.

---


total_orders = df.shape[0]
total_revenue = df['final_amount'].sum()
total_discount = df['discount_amount'].sum()
avg_discount_pct = df['discount_percentage'].mean()
avg_units = df['units_sold'].mean()
coupon_usage_rate = df['coupon_used'].mean() * 100

print(" Total Orders:", total_orders)
print(" Total Revenue (after discounts): ₹", round(total_revenue, 2))
print(" Total Discounts Given: ₹", round(total_discount, 2))
print(" Avg. Discount %:", round(avg_discount_pct, 2), "%")
print(" Avg. Units per Order:", round(avg_units, 2))
print(" Coupon Usage Rate:", round(coupon_usage_rate, 2), "%")


## Coupon ROI Analysis

Let’s calculate the **average final revenue generated per ₹1 of discount** given for each coupon type — an important metric for measuring marketing effectiveness.

---

In [ ]:
roi_df = df.groupby('coupon_type')[['final_amount', 'discount_amount']].sum()
roi_df['ROI_per_discount_rupee'] = roi_df['final_amount'] / roi_df['discount_amount']
roi_df = roi_df.sort_values(by='ROI_per_discount_rupee', ascending=False)

print(" ROI (₹ of revenue per ₹1 of discount):")
display(roi_df[['ROI_per_discount_rupee']].round(2))


In [ ]:
##  Best Coupon Types per Product Category

Let’s find the **most profitable coupon type** for each product category, based on average final spend per order.

---

In [ ]:
combo = df.groupby(['product_category', 'coupon_type'])['final_amount'].mean().reset_index()
best_combo = combo.sort_values(['product_category', 'final_amount'], ascending=[True, False]).drop_duplicates('product_category')
best_combo = best_combo.sort_values('final_amount', ascending=False)

print(" Best Coupon Type per Product Category:")
display(best_combo.reset_index(drop=True))


In [ ]:
## Business Intelligence Highlights

-  **Flat coupons** offer the highest ROI across categories.
-  **Groceries + Clothing** dominate in volume and spend.
-  **East + Central regions** show highest engagement — invest in those geos.
-  **Peak sales months**: November & December (leverage festive season).
-  **Buy1Get1** is overused but less ROI efficient in some categories.
-  High-income users spend more *without needing higher discounts*.

> If this was a product? It would be a **coupon optimizer tool** for D2C brands and ecommerce retailers.  
> If this was an app? It would show **best coupons per user** and **predict sales lift**.

---


In [ ]:
#  Final Save for App/Product Use
df.to_csv("final_coupon_analysis_product_ready.csv", index=False)
print(" Final dataset saved as 'final_coupon_analysis_product_ready.csv'")


In [ ]:

> Built with ❤️ by Ashish Sahu  
